# Prompt Chaining Examples

## Example 1: Research-Paper Digest → Quiz Builder

In [1]:
paper='This paper introduces a simple algorithm. It is easy to implement.'

def summarize(text):
    return text.split('.')[0] + '.'

def make_mcq(summary):
    return f"What does the paper introduce? A) {summary}"

def review(mcq):
    return '✅ ready' if mcq else '❌'

summary = summarize(paper)
mcq = make_mcq(summary)
status = review(mcq)
print('MCQ:', mcq)
print('Status:', status)


MCQ: What does the paper introduce? A) This paper introduces a simple algorithm.
Status: ✅ ready


## Example 2: Raw Marks → Dashboard

In [2]:
data=[{'name':'Ann','marks':55},{'name':'Ben','marks':35},{'name':'Cat','marks':75}]

def analyze(data):
    safe=[d for d in data if d['marks']>=40]
    risk=[d for d in data if d['marks']<40]
    return {'safe_count':len(safe),'risk_count':len(risk),'at_risk':risk}

def make_emails(info):
    return [f"Email to {d['name']}: please improve" for d in info['at_risk']]

info=analyze(data)
emails=make_emails(info)
print('Info:', info)
print('Emails:', emails)


Info: {'safe_count': 2, 'risk_count': 1, 'at_risk': [{'name': 'Ben', 'marks': 35}]}
Emails: ['Email to Ben: please improve']


## Example 3: Bug Report → Patch Suggestion

In [3]:
log='Exception at line 10'

def extract(log):
    return log

def guess_cause(text):
    return 'Off by one'

def propose_patch(cause):
    return 'Fix index'

text=extract(log)
cause=guess_cause(text)
patch=propose_patch(cause)
print('Cause:', cause)
print('Patch:', patch)


Cause: Off by one
Patch: Fix index


## Example 4: Lecture Notes → MCQs

In [4]:
notes='Recursion is a function calling itself.'

def objectives(text):
    return ['Define recursion']

def mcq(obj):
    return f'Recursion is? A) {obj[0]}'

objs=objectives(notes)
mcq_text=mcq(objs)
print('MCQ:', mcq_text)


MCQ: Recursion is? A) Define recursion
